# Introduction to Jupyter Notebooks
This lesson will introduce the Jupyter Notebook interface. We will use the interface to run and write, yes, write, some Python code for text data analysis.

By the end of this lesson, learners should be able to:
1. Explain the difference between markdown and code blocks in Jupyter Notebooks
2. Execute pre-written Python code to analyze newspaper text
3. Modify Python code to change the settings of the analysis

## What is this Jupyter Notebook thing?

Jupyter Notebooks are effectively made up of "cells". We can start by thinking of each cell being equivalent to a paragraph on a page. There is an order in which paragraphs and cells appear, and that order matters. In Jupyter Notebooks, the cells come in two flavors and a single notebook (like the one we are working in now) with have both types of cells. 
+ The first is called "markdown", which is text, like you are reading now. We can use some syntax in the text to format the cells in particular ways. For example, we can create italic text by using the underscore symbol ("\_") at the beginning and ending of the text we want to italicize. So when we write "\_italic\_" in a markdown block, it will show up as _italic_.
+ The second kind of cell is a "code" cell, that contains computer code in a language like Python or R. This is where the fun comes in.

**Do some markdown stuff?**

In [1]:
print("Collections as Data")

Collections as Data


## So what is Python then?

Add brief explantion of python?

In [ ]:
print("Hello World")

**talk about what we are going to do**

In [ ]:
# import stuff and run it

get the data and roadmap where we want to go. Maybe whiteboard the entire process?

In [ ]:
# download data and do a little reality check

some word counting before removing stopwords

In [ ]:
# word count and point out "the"

removal of stopwords and maybe single characters?

In [ ]:
# load corpora, remove stopwords, and look at results of count